<a href="https://colab.research.google.com/github/UAMCAntwerpen/2040FBDBIC/blob/master/Topic_04/Protein_ligand_MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Protein-ligand molecular dynamics (MD)

## Initial setup

The first thing you should do, is make a copy of this `Google Colab` session. This will allow you to make changes to code and this will be required in a later stage of this workbook. You can make a copy by going to *File* > *Save a copy in Drive*. This brings you to the new session and you are allowed to close the initial page.

## Introduction

In general, MD simulations rely on

1. a set of atomic coordinates of all atoms on a simulation box and
2. a set of force field parameters that describes the interaction energies between atoms.


In this notebook, we will simulate <a href="https://www.rcsb.org/structure/3HTB" target="_blank"> PDB entry 3HTB</a>. This is 2-propylphenol (the ligand) in complex with T4 lysozyme with the L99A and M102Q mutations (the protein). Therefore, in terms of inputs, we will need two `pdb` files that describe the coordinates of both molecules:

1. a `pdb` file of the protein
2. a `pdb` file of the ligand

To build our simulation box, we will use the <a href="https://ambermd.org/tutorials/pengfei/index.php" target="_blank">LEaP software</a>. The`LEaP` program is a program that allows one to convert and merge between many chemical structure file types (for example, `pdb` and `mol2` types - see <a href="https://uamcantwerpen.github.io/2040FBDBIC/Topic_01.html" target="_blank">Topic 1</a> of this course), and the Amber model parameter file types such as `lib`, `prepi`, `parm.dat`, and `frcmod`. Each of the parameter files contains pieces of information needed for constructing a simulation, whether for energy minimization or molecular dynamics. LEaP functions within a larger workflow described in the original <a href="https://ambermd.org/doc12/Amber20.pdf" download>Amber manual</a>. 

#### Ligand topology

To build ligand topology we will use **General AMBER Force Field** (<a href="http://ambermd.org/antechamber/gaff.html" target="_blank">GAFF</a>) for the ligand, and the **Open Force Field Toolkit** (<a href="3HTB" target="_blank">OpenFF</a>) for the protein. **GAFF** is compatible with the **AMBER** force field and it has parameters for almost all the organic molecules made of C, N, O, H, S, P, F, Cl, Br and I. As a complete force field, **GAFF** is suitable for study of a great number of molecules in an automatic fashion.

#### Protein topology

The **Open Force Field Toolkit**, built by the <a href="https://openforcefield.org/" target="_blank">Open Force Field Initiative</a>, is a Python toolkit for the develoment and application of modern molecular mechanics force fields based on direct chemical perception and rigorous statistical parameterization methods. This force field will be used to build a topology for the protein system.

## Acknowledgment

This notebook is entirely based on the <a href="https://github.com/pablo-arantes/Making-it-rain" target="_blank">excellent notebook of Pedro Arantes</a>.

## Setting up the environment for MD calculation

Firstly, we need to install all necessary libraries and packages for our simulation. The main packages we will be installing are:

1. <a href="https://docs.conda.io/en/latest/miniconda.html" target="_blank">Anaconda</a> (to create a Python environment in which other packages can be installed)
2. <a href="https://openmm.org/" target="_blank">OpenMM</a> (the protein forcefield and MD code)
3. <a href="https://amber-md.github.io/pytraj/latest/index.html" target="_blank">PyTraj</a> (to post-process and analyse MD trajectories)
4. <a href="https://pypi.org/project/py3Dmol/" target="_blank">py3Dmol</a> (to visualise molecules in a Google Colab environment)
5. <a href="https://numpy.org/" target="_blank">Numpy</a> (for numerical calculations)
6. <a href="https://matplotlib.org/" target="_blank">Matplotlib</a> (for plotting)
7. <a href="https://ambermd.org/AmberTools.php" target="_blank">AmberTools</a> (for the protein forcefield and box creation)

In [ ]:
#@title Install dependencies
#@markdown It will take a few minutes, please, drink a coffee and wait ;-)
%%capture
import sys
!pip -q install py3Dmol 2>&1 1>/dev/null
!pip install --upgrade MDAnalysis 2>&1 1>/dev/null
# !pip install git+https://github.com/pablo-arantes/biopandas 2>&1 1>/dev/null
# !pip install rdkit-pypi
!pip install Cython
# !git clone https://github.com/pablo-arantes/ProLIF.git
# prolif1 = "cd /content/ProLIF"
# prolif2 = "sed -i 's/mdanalysis.*/mdanalysis==2.0.0/' setup.cfg"
# prolif3 = "pip install ."

# original_stdout = sys.stdout # Save a reference to the original standard output

# with open('prolif.sh', 'w') as f:
#    sys.stdout = f # Change the standard output to the file we created.
#    print(prolif1)
#    print(prolif2)
#    print(prolif3)
#    sys.stdout = original_stdout # Reset the standard output to its original value

# !chmod 700 prolif.sh 2>&1 1>/dev/null
# !bash prolif.sh >/dev/null 2>&1

# Install conda
!wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh 
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
# !rm -r Miniconda3-latest-Linux-x86_64.sh /content/ProLIF prolif.sh
!conda install -y -q -c conda-forge openmm=7.6 python=3.8 pdbfixer 2>&1 1>/dev/null
# !conda install -c conda-forge openbabel --yes 2>&1 1>/dev/null
!conda install -c conda-forge ambertools --yes 2>&1 1>/dev/null
!conda install -c conda-forge parmed  --yes 2>&1 1>/dev/null
!conda install -c conda-forge openff-toolkit --yes 2>&1 1>/dev/null

# Load dependencies
sys.path.append('/usr/local/lib/python3.8/site-packages/')
# from openmm import app, unit
# from openmm.app import HBonds, NoCutoff, PDBFile
# from openff.toolkit.topology import Molecule, Topology
# from openff.toolkit.typing.engines.smirnoff import ForceField
# from openff.toolkit.utils import get_data_file_path
# import parmed as pmd
# from biopandas.pdb import PandasPdb
# import openmm as mm
# from openmm import *
# from openmm.app import *
# from openmm.unit import *
import os
import urllib.request  
# import py3Dmol
# import pytraj as pt
import platform
# !wget  https://raw.githubusercontent.com/openforcefield/openff-forcefields/master/openforcefields/offxml/openff_unconstrained-2.0.0.offxml 2>&1 1>/dev/null


## Using Google Drive to store simulation data

Google Colab does not allow users to keep data on their computing nodes. However, we can use `Google Drive` to read, write, and store our simulations files.

When `Google Drive` has been mounted, it is available from the `/content/drive/` folder.

#### Mount Google Drive

In [ ]:
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

#### Check if you have the required GPU node

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
#@title Downloading the pdb files of protein and ligand
#@markdown Specify which ligand:

# Download
ligand = "ligand1"  #@param ["ligand1", "ligand2", "ligand3", "ligand4"]
ligand = ligand + ".pdb"
cmd = "wget https://raw.githubusercontent.com/UAMCAntwerpen/2040FBDBIC/master/Topic_04/protein-ligand-md/%s" % (ligand)
os.system(cmd)
!wget https://raw.githubusercontent.com/UAMCAntwerpen/2040FBDBIC/master/Topic_04/protein-ligand-md/protein.pdb

# Make the directory if not already existing
![ -d /content/drive/MyDrive/protein_ligand ] || mkdir /content/drive/MyDrive/protein_ligand

# Move the downloaded files to the new directory; rename the ligand to "ligand.pdb"
cmd = "mv %s /content/drive/MyDrive/protein_ligand/ligand.pdb" % (ligand)
os.system(cmd)
!mv protein.pdb /content/drive/MyDrive/protein_ligand/protein.pdb

# Show the contents of the new directory
!ls /content/drive/MyDrive/protein_ligand

## Setting up the protein and ligand

#### First start with the ligand

In [ ]:
# Derive GAFF2 parameters
cmd =  "antechamber -fi pdb -i /content/drive/MyDrive/protein_ligand/ligand.pdb "
cmd += "-fo prepi -o /content/drive/MyDrive/protein_ligand/ligand.prepi "
cmd += "-c bcc -nc 0 "
cmd += "-rn LIG -pf yes"
os.system(cmd)

cmd =  "parmchk2 -f prepi "
cmd += "-i /content/drive/MyDrive/protein_ligand/ligand.prepi "
cmd += "-o /content/drive/MyDrive/protein_ligand/ligand.frcmod"
os.system(cmd)

cmd =  "antechamber -fi pdb -i /content/drive/MyDrive/protein_ligand/ligand.pdb "
cmd += "-fo mol2 -o /content/drive/MyDrive/protein_ligand/ligand.mol2 "
cmd += "-c bcc -nc 0 "
cmd += "-rn LIG -pf yes"
os.system(cmd)

#### Now the box will be prepared

In [ ]:
# Create a tleap input file
inp = "/content/drive/MyDrive/protein_ligand/tleap.inp"
f = open(inp, "w")

f.write("source leaprc.protein.ff19SB\n")
f.write("source leaprc.water.opc\n")
f.write("source leaprc.gaff2\n")
f.write("loadamberprep /content/drive/MyDrive/protein_ligand/ligand.prepi\n")
f.write("loadamberparams /content/drive/MyDrive/protein_ligand/ligand.frcmod\n")

f.write("protein = loadpdb /content/drive/MyDrive/protein_ligand/protein.pdb\n")
f.write("ligand = loadmol2 /content/drive/MyDrive/protein_ligand/ligand.mol2\n")
f.write("complex = combine { protein ligand }\n")
f.write("check complex\n")

f.write("solvateBox complex OPCBOX 10 iso\n")
f.write("addions2 complex Cl- 0\n")
f.write("addions2 complex Na+ 0\n")

f.write("savePDB complex /content/drive/MyDrive/protein_ligand/complex.pdb\n")
prm = "/content/drive/MyDrive/protein_ligand/complex.prmtop"
crd = "/content/drive/MyDrive/protein_ligand/complex.ncrst"
f.write("saveAmberParm complex %s %s\n" % (prm, crd))
f.close()

log = "/content/drive/MyDrive/protein_ligand/tleap.log"
os.system("tleap -f %s 2>&1 1>%s" % (inp, log))

## Let's take a look on our simulation box:

In [ ]:
#@title Show 3D structure:
import warnings
warnings.filterwarnings('ignore')
import py3Dmol

color = "gray" #@param ["gray", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
show_ligand = True #@param {type:"boolean"}
show_box = True #@param {type:"boolean"}
box_opacity = 0.3 #@param {type:"slider", min:0, max:1, step:0.1}

pdb = "/content/drive/MyDrive/protein_ligand/complex.pdb"
def show_pdb(show_sidechains=False, show_mainchains=False, show_ligand = False, show_box = False, color="rainbow"):
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb,'r').read(),'pdb')

  if color == "gray":
    view.setStyle({'cartoon':{}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})

  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})  
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  
  if show_box:
    view.addSurface(py3Dmol.SAS, {'opacity': box_opacity, 'color':'white'})
  
  if show_ligand:
    HP = ['LIG']
    view.addStyle({'and':[{'resn':HP}]},
                       {'stick':{'colorscheme':'greenCarbon','radius':0.3}})
    view.setViewStyle({'style':'outline','color':'black','width':0.1})

  view.zoomTo()
  return view


show_pdb(show_sidechains, show_mainchains, show_ligand, show_box, color).show()

## Equilibrating the simulation box

Proper MD equilibration protocol is designed to equilibrate both temperature and pressure throughout the simulation box while preserving the protein experimental conformation. In addition, we also allow the solvent to accomodate around the protein, creating proper solvation layers.

Below, we will set up the MD equilibration parameters, such as temperature, pressure and the desired simulation time. We will define the force constant used to restraint protein heavy-atoms in place and the frequency at which we want to save atomic coordinates in a trajectory file (.dcd).

After you are done, you can run the next two cells to equilibrate your system.

In [ ]:
#@title #### Parameters for the MD equilibration protocol:
workDir = "/content/drive/MyDrive/protein_ligand/"
top = os.path.join(workDir, "complex.prmtop")
crd = os.path.join(workDir, "complex.ncrst")
pdb = os.path.join(workDir, "complex.pdb")

#@markdown Number of simulation steps:
Minimization_steps = "10000" #@param ["1000", "5000", "10000", "20000", "50000", "100000"]

#@markdown Simulation time (in ns) and integration time (in fs): 
Time = "1" #@param {type:"string"}
stride_time_eq = Time
Integration_timestep = "2" #@param ["0.5", "1", "2", "3", "4"]
dt_eq = Integration_timestep

#@markdown Temperature (in K) and pressure (in bar):
Temperature = 298 #@param {type:"string"}
temperature_eq = Temperature
Pressure = 1 #@param {type:"string"}
pressure_eq = Pressure

#@markdown Position restraints force constant (in kJ/mol): 
Force_constant = 700 #@param {type:"slider", min:0, max:2000, step:100}

#@markdown Frequency to write the trajectory file (in ps): 
Write_the_trajectory = "100" #@param ["10", "100", "200", "500", "1000"]
write_the_trajectory_eq = Write_the_trajectory

#@markdown Frequency to write the log file (in ps): 
Write_the_log = "100" #@param ["10", "100", "200", "500", "1000"]
write_the_log_eq = Write_the_log



In [ ]:
#@title Runs an equilibration MD simulation (NPT ensemble)
#@markdown Now, let's equilibrate our system!

###########################################
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import pytraj as pt

from sys import stdout, exit, stderr
import os, math, fnmatch

#############################################
# Defining MD simulation parameters

jobname = os.path.join(workDir, "prot_lig_equil")
coordinatefile = crd
pdbfile = pdb
topologyfile = top

time_ps = float(Time)*1000
simulation_time = float(time_ps)*picosecond		# in ps
dt = int(dt_eq)*femtosecond					
temperature = float(temperature_eq)*kelvin
savcrd_freq = int(write_the_trajectory_eq)*picosecond
print_freq  = int(write_the_log_eq)*picosecond

pressure	= float(pressure_eq)*bar

restraint_fc = int(Force_constant) # kJ/mol

nsteps  = int(simulation_time.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nprint  = int(print_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nsavcrd = int(savcrd_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))

#############################################
# Defining functions to use below:
def backup_old_log(pattern, string):
	result = []
	for root, dirs, files in os.walk("./"):
		for name in files:
			if fnmatch.fnmatch(name, pattern):

				try:
					number = int(name[-2])
					avail = isinstance(number, int)
					#print(name,avail)
					if avail == True:
						result.append(number)
				except:
					pass

	if len(result) > 0:
		maxnumber = max(result)
	else:
		maxnumber = 0

	backup_file = "\#" + string + "." + str(maxnumber + 1) + "#"
	os.system("mv " + string + " " + backup_file)
	return backup_file

def restraints(system, crd, fc, restraint_array):

	boxlx = system.getDefaultPeriodicBoxVectors()[0][0].value_in_unit(nanometers)
	boxly = system.getDefaultPeriodicBoxVectors()[1][1].value_in_unit(nanometers)
	boxlz = system.getDefaultPeriodicBoxVectors()[2][2].value_in_unit(nanometers)

	if fc > 0:
		# positional restraints for all heavy-atoms
		posresPROT = CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2;')
		posresPROT.addPerParticleParameter('k')
		posresPROT.addPerParticleParameter('x0')
		posresPROT.addPerParticleParameter('y0')
		posresPROT.addPerParticleParameter('z0')
  
		for atom1 in restraint_array:
			atom1 = int(atom1)
               
			xpos  = crd.positions[atom1].value_in_unit(nanometers)[0]
			ypos  = crd.positions[atom1].value_in_unit(nanometers)[1]
			zpos  = crd.positions[atom1].value_in_unit(nanometers)[2]

			posresPROT.addParticle(atom1, [fc, xpos, ypos, zpos])
    
		system.addForce(posresPROT)

	return system
##############################################

#############################################
print("\n> Simulation details:\n")
print("\tJob name = " + jobname)
print("\tCoordinate file = " + str(coordinatefile))
print("\tPDB file = " + str(pdbfile))
print("\tTopology file = " + str(topologyfile))

print("\n\tSimulation_time = " + str(simulation_time))
print("\tIntegration timestep = " + str(dt))
print("\tTotal number of steps = " +  str(nsteps))

print("\n\tSave coordinates each " + str(savcrd_freq))
print("\tPrint in log file each " + str(print_freq))

print("\n\tTemperature = " + str(temperature))
print("\tPressure = " + str(pressure))
#############################################

print("\n> Setting the system:\n")

print("\t- Reading topology and structure file...")
prmtop = AmberPrmtopFile(topologyfile)
inpcrd = AmberInpcrdFile(coordinatefile)

print("\t- Creating system and setting parameters...")
nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
friction = 1.0
system = prmtop.createSystem(nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
                                          constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance)


print("\t- Applying restraints. Force Constant = " + str(Force_constant) + "kJ/mol")
pt_system = pt.iterload(coordinatefile, topologyfile)
pt_topology = pt_system.top
restraint_array = pt.select_atoms('!(:H*) & !(:WAT) & !(:Na+) & !(:Cl-) & !(:Mg+) & !(:K+)', pt_topology)

system = restraints(system, inpcrd, restraint_fc, restraint_array)

print("\t- Setting barostat...")
system.addForce(MonteCarloBarostat(pressure, temperature))

print("\t- Setting integrator...")
integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

print("\t- Energy minimization: " + str(Minimization_steps) + " steps")
simulation.minimizeEnergy(tolerance=10*kilojoule/mole, maxIterations=int(Minimization_steps))
print("\t-> Potential Energy = " + str(simulation.context.getState(getEnergy=True).getPotentialEnergy()))

print("\t- Setting initial velocities...")
simulation.context.setVelocitiesToTemperature(temperature)

#############################################
# Running equilibration on NPT ensemble

dcd_file = jobname + ".dcd"
log_file = jobname + ".log"
rst_file = jobname + ".rst"
prv_rst_file = jobname + ".rst"
pdb_file = jobname + ".pdb"

# Creating a trajectory file and reporters
dcd = DCDReporter(dcd_file, nsavcrd)
firstdcdstep = (nsteps) + nsavcrd
dcd._dcd = DCDFile(dcd._out, simulation.topology, simulation.integrator.getStepSize(), firstdcdstep, nsavcrd) # charmm doesn't like first step to be 0

simulation.reporters.append(dcd)
simulation.reporters.append(StateDataReporter(stdout, nprint, step=True, speed=True, progress=True, totalSteps=nsteps, remainingTime=True, separator='\t\t'))
simulation.reporters.append(StateDataReporter(log_file, nprint, step=True, kineticEnergy=True, potentialEnergy=True, totalEnergy=True, temperature=True, volume=True, speed=True))

print("\n> Simulating " + str(nsteps) + " steps...")
simulation.step(nsteps)

simulation.reporters.clear() # remove all reporters so the next iteration don't trigger them.


##################################
# Writing last frame information of stride
print("\n> Writing state file (" + str(rst_file) + ")...")
state = simulation.context.getState( getPositions=True, getVelocities=True )
with open(rst_file, 'w') as f:
	f.write(XmlSerializer.serialize(state))

last_frame = int(nsteps/nsavcrd)
print("> Writing coordinate file (" + str(pdb_file) + ", frame = " + str(last_frame) + ")...")
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open(pdb_file, 'w'))

print("\n> Finished!\n")

## Running a production MD simulation

Finally, we will proceed with the production simulation itself using the equilibrated system coordinates as input structure.

Note that we will use here a `rst` state file, which contains atomic velocities and positions from the last frame of the equilibration simulation, guaranteeing that our production simulation begins from a thermodynamically equilibrated system.


In [ ]:
#@title #### Parameters for the MD production protocol:
Equilibrated_PDB = 'prot_lig_equil.pdb'
State_file = 'prot_lig_equil.rst'

top = os.path.join(workDir, "complex.prmtop")

#@markdown Simulation time (in ns) and integration timestep (in fs): 
Simulation_time  = "2" #@param {type:"string"}
stride_time_prod = Simulation_time
# Number_of_strides = "1" #@param {type:"string"}
#nstride = Number_of_strides
Integration_timestep = "2" #@param ["0.5", "1", "2", "3", "4"]
dt_prod = Integration_timestep

#@markdown Temperature (in K) and pressure (in bar)
Temperature = 298 #@param {type:"string"}
temperature_prod = Temperature
Pressure = 1 #@param {type:"string"}
pressure_prod = Pressure

#@markdown Frequency to write the trajectory file (in ps): 
Write_the_trajectory = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_trajectory_prod = Write_the_trajectory

#@markdown Frequency to write the log file (in ps): 
Write_the_log = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_log_prod = Write_the_log


In [ ]:
#@title Runs a production MD simulation (NPT ensemble) after equilibration
#
###########################################
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *

from sys import stdout, exit, stderr
import os, math, fnmatch

#############################################
# Defining MD simulation parameters

jobname = os.path.join(workDir, 'prot_lig_prod')
coordinatefile = crd
pdbfile = os.path.join(workDir, Equilibrated_PDB)
topologyfile = top
equil_rst_file = os.path.join(workDir, State_file)


stride_time_ps = float(stride_time_prod)*1000
stride_time = float(stride_time_ps)*picosecond        
dt = int(dt_prod)*femtosecond					
temperature = float(temperature_prod)*kelvin
savcrd_freq = int(write_the_trajectory_prod)*picosecond
print_freq  = int(write_the_log_prod)*picosecond

pressure	= float(pressure_prod)*bar

simulation_time = stride_time
nsteps  = int(stride_time.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nprint  = int(print_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nsavcrd = int(savcrd_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
#############################################
# Defining functions to use below:
def backup_old_log(pattern, string):
	result = []
	for root, dirs, files in os.walk("./"):
		for name in files:
			if fnmatch.fnmatch(name, pattern):

				try:
					number = int(name[-2])
					avail = isinstance(number, int)
					#print(name,avail)
					if avail == True:
						result.append(number)
				except:
					pass

	if len(result) > 0:
		maxnumber = max(result)
	else:
		maxnumber = 0

	backup_file = "\#" + string + "." + str(maxnumber + 1) + "#"
	os.system("mv " + string + " " + backup_file)
	return backup_file
##############################################

#############################################
print("\n> Simulation details:\n")
print("\tJob name = " + jobname)
print("\tCoordinate file = " + str(coordinatefile))
print("\tPDB file = " + str(pdbfile))
print("\tTopology file = " + str(topologyfile))

print("\n\tSimulation_time = " + str(stride_time))
print("\tIntegration timestep = " + str(dt))
print("\tTotal number of steps = " +  str(nsteps))

print("\n\tSave coordinates each " + str(savcrd_freq))
print("\tSave checkpoint each " + str(savcrd_freq))
print("\tPrint in log file each " + str(print_freq))

print("\n\tTemperature = " + str(temperature))
print("\tPressure = " + str(pressure))
#############################################

print("\n> Setting the system:\n")

print("\t- Reading topology and structure file...")
prmtop = AmberPrmtopFile(topologyfile)
inpcrd = AmberInpcrdFile(coordinatefile)

print("\t- Creating system and setting parameters...")
nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
friction = 1.0
system = prmtop.createSystem(nonbondedMethod=nonbondedMethod,
                             nonbondedCutoff=nonbondedCutoff,
														 constraints=constraints,
														 rigidWater=rigidWater,
														 ewaldErrorTolerance=ewaldErrorTolerance)

print("\t- Setting barostat...")
system.addForce(MonteCarloBarostat(pressure, temperature))

print("\t- Setting integrator...")
integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
	simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

#############################################
dcd_file = jobname + "_1.dcd"
log_file = jobname + "_1.log"
rst_file = jobname + "_1.rst"
prv_rst_file = jobname + "_0.rst"
pdb_file = jobname + "_1.pdb"

print("\n> Loading previous state from equilibration > " + equil_rst_file + " <")
with open(equil_rst_file, 'r') as f:
	simulation.context.setState(XmlSerializer.deserialize(f.read()))
	currstep = 0
	currtime = currstep*dt.in_units_of(picosecond)
	simulation.currentStep = currstep
	simulation.context.setTime(currtime)
	print("> Current time: " + str(currtime) + " (Step = " + str(currstep) + ")")

	dcd = DCDReporter(dcd_file, nsavcrd)
	firstdcdstep = (currstep) + nsavcrd
	dcd._dcd = DCDFile(dcd._out, simulation.topology, simulation.integrator.getStepSize(), firstdcdstep, nsavcrd) # first step should not be 0

	simulation.reporters.append(dcd)
	simulation.reporters.append(StateDataReporter(stdout, nprint, step=True, speed=True, progress=True, totalSteps=(nsteps), remainingTime=True, separator='\t\t'))
	simulation.reporters.append(StateDataReporter(log_file, nprint, step=True, kineticEnergy=True, potentialEnergy=True, totalEnergy=True, temperature=True, volume=True, speed=True))

	print("\n> Simulating " + str(nsteps) + " steps...")
	simulation.step(nsteps)

	simulation.reporters.clear() # remove all reporters so the next iteration don't trigger them.


	##################################
	# Writing last frame information of stride
	print("\n> Writing state file (" + str(rst_file) + ")...")
	state = simulation.context.getState( getPositions=True, getVelocities=True )
	with open(rst_file, 'w') as f:
		f.write(XmlSerializer.serialize(state))

	last_frame = int(nsteps/nsavcrd)
	print("> Writing coordinate file (" + str(pdb_file) + ", frame = " + str(last_frame) + ")...")
	positions = simulation.context.getState(getPositions=True).getPositions()
	PDBFile.writeFile(simulation.topology, positions, open(pdb_file, 'w'))

print("\n> Finished!\n")

In [ ]:
#@title Align the trajectory and optionally remove waters

import MDAnalysis as mda
from MDAnalysis.analysis import align, rms

workDir = '/content/drive/MyDrive/protein_ligand'
Equilibrated_PDB = 'prot_lig_equil.pdb'
Jobname = "prot_lig_prod"

stride_traj = 1
Output_format = "dcd"
traj_save_freq = write_the_trajectory_prod
Remove_waters = "yes" #@param ["yes", "no"]
output_prefix = "1-1"

simulation_time_analysis = stride_time_ps
simulation_ns = float(stride_time / nanoseconds)
number_frames = int(simulation_time_analysis)/int(traj_save_freq)
number_frames_analysis = number_frames

nw_dcd = os.path.join(workDir, str(Jobname) + output_prefix + "_nw." + str(Output_format))
nw_pdb = os.path.join(workDir, str(Jobname) +  "_nw.pdb")
whole_pdb = os.path.join(workDir, str(Jobname) +  "_whole.pdb")
whole_dcd = os.path.join(workDir, str(Jobname) + output_prefix + "_whole." + str(Output_format))
template =  os.path.join(workDir, str(Jobname) + '_%s.dcd')
pdb = os.path.join(workDir, Equilibrated_PDB)

flist = [template % str(i) for i in range(1, 2)]
ref = [template % int(1)]

u1 = mda.Universe(os.path.join(workDir, "complex.prmtop"), flist)
u2 = mda.Universe(os.path.join(workDir, "complex.prmtop"), ref)

u2.trajectory[0] # set u2 to first frame

align.AlignTraj(u1, u2, select='name CA', in_memory=True).run()

with mda.Writer(whole_dcd, u1.select_atoms("all").n_atoms) as W:
  for ts in u1.trajectory[::1]:
      W.write(u1.select_atoms("all"))
whole = u2.select_atoms("all")
whole.write(whole_pdb)
traj_dcd_check = os.path.exists(whole_dcd)
traj = whole_dcd
pdb_ref = whole_pdb
traj_gaff = pt.load(traj, os.path.join(workDir, "complex.prmtop"))

nw = u1.select_atoms("not (resname WAT)")
if Remove_waters == "yes":
  with mda.Writer(nw_dcd, nw.n_atoms) as W:
    for ts in u1.trajectory[::1]:
        W.write(nw)
  not_waters = u2.select_atoms("not (resname WAT)")
  not_waters.write(nw_pdb)
  traj_dcd_check = os.path.exists(nw_dcd)
  traj = nw_dcd
  pdb_ref = nw_pdb
else:
  pass

traj_load = pt.load(traj, pdb_ref)
print(traj_load)

if traj_dcd_check == True:
  print("Trajectory concatenated successfully! :-)")
else:
  print("ERROR: Check your inputs! ")

In [ ]:
#@title Load, view and check the trajectory:
#@markdown This will take a few minutes. Another coffee would be great. :-)

import warnings
warnings.filterwarnings('ignore')

#py3dmol functions
class Atom(dict):
  def __init__(self, line):
    self["type"] = line[0:6].strip()
    self["idx"] = line[6:11].strip()
    self["name"] = line[12:16].strip()
    self["resname"] = line[17:20].strip()
    self["resid"] = int(int(line[22:26]))
    self["x"] = float(line[30:38])
    self["y"] = float(line[38:46])
    self["z"] = float(line[46:54])
    self["sym"] = line[76:78].strip()

  def __str__(self):
    line = list(" " * 80)
    line[0:6] = self["type"].ljust(6)
    line[6:11] = self["idx"].ljust(5)
    line[12:16] = self["name"].ljust(4)
    line[17:20] = self["resname"].ljust(3)
    line[22:26] = str(self["resid"]).ljust(4)
    line[30:38] = str(self["x"]).rjust(8)
    line[38:46] = str(self["y"]).rjust(8)
    line[46:54] = str(self["z"]).rjust(8)
    line[76:78] = self["sym"].rjust(2)
    return "".join(line) + "\n"
        
class Molecule(list):
  def __init__(self, file):
    for line in file:
      if "ATOM" in line or "HETATM" in line:
        self.append(Atom(line))
            
    def __str__(self):
      outstr = ""
      for at in self:
        outstr += str(at)
      return outstr

if number_frames_analysis > 10:
  stride_animation = number_frames_analysis/10
else:
  stride_animation = 1

u = mda.Universe(pdb_ref, traj) 

# Write out frames for animation
protein = u.select_atoms('not (resname WAT)')
i = 0
for ts in u.trajectory[0:len(u.trajectory):int(stride_animation)]: 
    if i > -1:
        with mda.Writer('' + str(i) + '.pdb', protein.n_atoms) as W:
            W.write(protein)
    i = i + 1
# Load frames as molecules
molecules = []
for i in range(int(len(u.trajectory)/int(stride_animation))):
    with open('' + str(i) + '.pdb') as ifile:
        molecules.append(Molecule(ifile))

models = ""
for i in range(len(molecules)):
  models += "MODEL " + str(i) + "\n"
  for j,mol in enumerate(molecules[i]):
    models += str(mol)
  models += "ENDMDL\n"
#view.addModelsAsFrames(models)

# Animation
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(models)
for i, at in enumerate(molecules[0]):
    default = {"cartoon": {'color': 'spectrum'}}
    view.setViewStyle({'style':'outline','color':'black','width':0.1})
    view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))
    HP = ['LIG']
    view.setStyle({"model":-1,'and':[{'resn':HP}]},{'stick':{'radius':0.3}})
view.zoomTo()
view.animate({'loop': "forward"})
view.show()

## Analysis

Although visualizing your trajectory can be quite useful, sometimes you also want more quantitative data.

Analyses of MD trajectories vary a lot and we do not intend to cover it all here. However, one can make use of `MDanalysis` or `PyTraj` to easily analyze simulations. 

Below, you can find a few examples of code snippets that can help you to shed some light on your simulation behavior.

In [ ]:
import numpy as np
import MDAnalysis as mda
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import seaborn as sb
from statistics import mean, stdev
from pytraj import matrix
from matplotlib import colors
from IPython.display import set_matplotlib_formats
import pandas as pd

#@title Compute distance between the ligand and specific residues
#@markdown **Provide output file names below:** 
Output_name = 'distance_select' #@param {type:"string"}
#@markdown **Type the number of residues separated by commas and without spaces (1,2,3...):**
Residues = '78,84,85' #@param {type:"string"}

mask = ":LIG :" + str(Residues)
dist = pt.distance(traj_load, mask)
print("Selected residues = " + Residues + "\n")
dist_mean = mean(dist)
dist_stdev = stdev(dist)
print("Distance Average = " + str("{:.2f}".format(dist_mean)) + " \u00B1 " + str("{:.2f}".format(dist_stdev)) + " Å")

time = len(dist)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

# Plotting:
ax = plt.plot(time_array, dist, alpha=1, color = 'magenta', linewidth = 1.0)
plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Distance [$\AA$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(dist)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title Compute RMSD of protein's CA atoms
#@markdown **Provide output file names below:** 
Output_name = 'rmsd_ca' #@param {type:"string"}


rmsd = pt.rmsd(traj_load, ref = 0, mask = "@CA")

time = len(rmsd)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

# Plotting:
ax = plt.plot(time_array, rmsd, alpha=0.6, color = 'blue', linewidth = 1.0)
plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("RMSD [$\AA$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(rmsd)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title Plot RMSD as a distribution

#@markdown **Provide output file names below:** 
Output_name = 'rmsd_dist' #@param {type:"string"}

ax = sb.kdeplot(rmsd, color="blue", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('RMSD [$\AA$]', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title Compute radius of gyration of protein's CA atoms

#@markdown **Provide output file names below:** 
Output_name = 'radius_gyration' #@param {type:"string"}

radgyr = pt.radgyr(traj_load, mask = "@CA")
time = len(rmsd)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

# Plotting:
plt.plot(time_array, radgyr, alpha=0.6, color = 'green', linewidth = 1.0)
plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Radius of gyration ($\AA$)", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(radgyr)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title Plot radius of gyration as a ditribution

#@markdown **Provide output file names below:** 
Output_name = 'radius_gyration_dist' #@param {type:"string"}

ax = sb.kdeplot(radgyr, color="green", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('Radius of gyration ($\AA$)', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title Compute RMSF of protein's CA atoms

#@markdown **Provide output file names below:** 
Output_name = 'rmsf_ca' #@param {type:"string"}


rmsf = pt.rmsf(traj_load, "@CA")
bfactor = pt.bfactors(traj_load, byres=True)

# Plotting:
plt.plot(rmsf[:,1], alpha=1.0, color = 'red', linewidth = 1.0)

plt.xlabel("Residue", fontsize = 14, fontweight = 'bold')
plt.ylabel("RMSF ($\AA$)", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.xlim(0, len(rmsf[:-1]))

#plt.xticks(np.arange(min(rmsf[:1]), max(rmsf[:1])))
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(rmsf)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title 2D RMSD

#@markdown **Provide output file names below:** 
Output_name = '2D_rmsd' #@param {type:"string"}

last_frame = len(time_array)

stride_ticks_f = (last_frame)/5
ticks_frame = np.arange(0,(len(time_array) + float(stride_ticks_f)), float(stride_ticks_f))
a = ticks_frame.astype(float)
stride_ticks_t = (simulation_ns)/5
tick_time = np.arange(0,(float(simulation_ns) + float(stride_ticks_t)), float(stride_ticks_t))
b = tick_time.astype(float)

mat1 = pt.pairwise_rmsd(traj_load, mask="@CA", frame_indices=range(int(number_frames_analysis)))


ax = plt.imshow(mat1, cmap = 'PRGn', origin='lower', interpolation = 'bicubic')
plt.title('2D RMSD')
plt.xlabel('Time (ns)', fontsize = 14, fontweight = 'bold')
plt.ylabel('Time (ns)', fontsize = 14, fontweight = 'bold')
# plt.xticks(fontsize = 12)
# plt.yticks(fontsize = 12)
plt.xticks(a, b.round(decimals=3), fontsize = 12)
plt.yticks(a, b.round(decimals=3), fontsize = 12)
# plt.xlim(0, a[-1])
# plt.ylim(0, a[-1])

cbar1 = plt.colorbar()
cbar1.set_label("RMSD ($\AA$)", fontsize = 14, fontweight = 'bold')


plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(mat1)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
import os

#@title MM-PBSA method to calculate the binding free energy
#@markdown **Important:** We will now calculate the interaction energy and solvation free energy for the complex, receptor and ligand and average the results to obtain an estimate of the binding free energy. Please note that we will not perform a calculation of the entropy contribution to binding and so strictly speaking our result will not be a true free energy but could be used to compare against similar systems. We will carry out the binding energy calculation using both the MM-GBSA method and the MM-PBSA method for comparison.
igb = "2" 
mbondi = 'mbondi2'

Salt_concentration = '0.15'
fold_MMPBSA = "MMPBSA_igb_" + igb 
Output_name = 'FINAL_RESULTS_MMPBSA'

final_mmpbsa = os.path.join(workDir, Output_name)


f = open("mmpbsa.in", "w")
f.write("&general \n")
#f.write("  startframe=1,\n")
f.write("  endframe=%d,\n" % (number_frames_analysis))
f.write("  interval=1,\n")
f.write("  verbose=2,\n")
f.write("  keep_files=2,\n")
f.write("  strip_mask=:WAT:Na+:Cl-:Mg+:K+,\n")
f.write("/\n")
f.write("&gb \n")
f.write("  igb=2,\n")
f.write("  saltconc=0.15,\n")
f.write("/\n")
f.write("&pb \n")
f.write("  istrng=0.15,\n")
f.write("  inp=2,\n")
f.write("  radiopt=0,\n")
f.write("  prbrad=1.4,\n")
f.write("/\n")
f.close()

amberhome = "source /usr/local/amber.sh\n"

ante_MMPBSA =  "ante-MMPBSA.py \\\n"
ante_MMPBSA += " -p " + os.path.join(workDir, "complex.prmtop") + "\\\n"
ante_MMPBSA += " -c com.prmtop -r rec.prmtop -l ligand.prmtop \\\n"
ante_MMPBSA += " -s :WAT:Na+:Cl-:Mg+:K+ -n :LIG  \\\n"
ante_MMPBSA += "--radii " + str(mbondi) 

MMPBSA =  "MMPBSA.py \\\n"
MMPBSA += " -O \\\n"
MMPBSA += " -i mmpbsa.in \\\n"
MMPBSA += " -o " + str(final_mmpbsa) +  ".dat \\\n" 
MMPBSA += " -sp " + os.path.join(workDir, "complex.prmtop") + " \\\n"
MMPBSA += " -cp com.prmtop \\\n"
MMPBSA += " -rp rec.prmtop \\\n"
MMPBSA += " -lp ligand.prmtop \\\n"
MMPBSA += " -y " + str(whole_dcd) + "\n"

mkdir = "mkdir " + os.path.join(workDir, fold_MMPBSA) + "\n"

mv =  "mv "
mv += " _MMPBSA*"
mv += " com.prmtop"
mv += " rec.prmtop"
mv += " ligand.prmtop"
mv += " reference.frc"
mv += " MMPBSA.log"
mv += " mmpbsa.in " + os.path.join(workDir, fold_MMPBSA)

original_stdout = sys.stdout # Save a reference to the original standard output

with open('run_MMPBSA.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(amberhome)
    print(ante_MMPBSA)
    print(MMPBSA)
    print(mkdir)
    print(mv)
    sys.stdout = original_stdout # Reset the standard output to its original value

os.system("bash ./run_MMPBSA.sh 2>&1 1>MMPBSA.log")

f_mmpbsa = open(final_mmpbsa + '.dat', 'r')
file_contents = f_mmpbsa.read()
print(file_contents)
f_mmpbsa.close()

In [ ]:
#@title Cleanup all remaining files

os.system("rm sqm.*")
os.system("rm run_MMPBSA.sh")
os.system("rm leap.log")
os.system("rm *.pdb*")
os.system("rm -rf /content/drive/MyDrive/protein_ligand/")

# Repeat with other ligands

Now you can compare the MM-PBSA results with those from other ligands. Make a copy of the results above, and [navigate to starting cell](#scrollTo=Jh8C2zpu536a) to repeat the process.
